<p align="center">
  <img src="assets/pokemon-center.gif" alt="PKCenter" width="300"/>
</p>

In [60]:
# Checkpoint to begin loading the Showdown data set

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats as sp

gen5_types_df = pd.read_csv("./dataset/pokemon-fandom/pokemon_gen5_types_fixed.csv")
gen5_stats_df = pd.read_csv(r"dataset/bulbagarden/pokemon_gen5_stats.csv")
showdown_df = pd.read_csv("showdown.csv")
unique_pokemons_df = pd.read_csv("unique_pokemons.csv")
gen5ou_usage_df = pd.read_csv("gen5ou_usage.csv")

## Paired Observation test

In this test, we aim to determine if there will be a difference in pokemon usage rates in Generation 5 OU between the year 2015 and the year 2025. To do this, we will first be getting the differences of both data sets.

In [61]:
gen5ou_usage_df

,Rank_2015,Pokemon,Usage%_2015,Raw_2015,%_2015,Real_2015,%.1_2015,Rank_2025,Usage%_2025,Raw_2025,%_2025,Real_2025,%.1_2025
0,62.0,Abomasnow,2.18801,81.0,2.188,72.0,2.478,37.0,3.97171,3409.0,3.972,2979.0,4.382
1,266.0,Abra,0.05402,2.0,0.054,2.0,0.069,NaN,NaN,NaN,NaN,NaN,NaN
2,173.0,Absol,0.24311,9.0,0.243,8.0,0.275,176.0,0.18641,160.0,0.186,130.0,0.191
3,291.0,Accelgor,0.05402,2.0,0.054,2.0,0.069,169.0,0.20855,179.0,0.209,149.0,0.219
4,74.0,Aerodactyl,1.70178,63.0,1.702,57.0,1.962,76.0,1.21050,1039.0,1.211,968.0,1.424
...,...,...,...,...,...,...,...,...,...,...,...,...,...
506,NaN,Zigzagoon,NaN,NaN,NaN,NaN,NaN,435.0,0.00233,2.0,0.002,1.0,0.001
507,82.0,Zoroark,1.43166,53.0,1.432,18.0,0.620,86.0,1.01361,870.0,1.014,289.0,0.425
508,256.0,Zorua,0.05402,2.0,0.054,0.0,0.000,434.0,0.00233,2.0,0.002,0.0,0.000
509,360.0,Zubat,0.02701,1.0,0.027,1.0,0.034,NaN,NaN,NaN,NaN,NaN,NaN


### Pre-processing

Before we proceed with testing, we first would like to discern if the differences in the usage rates of pokemons are normal. This would determine the paired observation test we would be performing. To do this, we create a temporary dataframe containing the usgae rates of pokemon from both years and assign the difference to a Difference column and perform the Shapiro Wilk Test of normality.

The Shapiro Wilks Test of Normality is a test that determines if a given list of values is normally distributed or not. 

The hypothesis is as follows:
* $H_0$: The data is normally distributed
* $H_a$: The data is not normally distributed

We set a significance level of $a = 0.05$. As such, we will reject the null hypothesis if the $p$ value reaches below this threshold.

In [62]:
usage_diff = gen5ou_usage_df[["Pokemon", "Usage%_2015", "Usage%_2025"]].fillna(0)
usage_diff.loc[:, "Difference"] = usage_diff["Usage%_2025"] - usage_diff["Usage%_2015"]

In [63]:
usage_diff["Difference"]

0      1.78370
1     -0.05402
2     -0.05670
3      0.15453
4     -0.49128
        ...   
506    0.00233
507   -0.41805
508   -0.05169
509   -0.02701
510    0.00117
Name: Difference, Length: 511, dtype: float64

In [64]:
stat, p = sp.shapiro(usage_diff["Difference"])
print(f"Shapiro-Wilk test: stat={stat}, p={p}")
if p > 0.05:
    print("Likely normal")
else:
    print("Likely not normal")

Shapiro-Wilk test: stat=0.46864922432647227, p=2.7047915139577904e-36
Likely not normal


Based on the test, we get a $p$ value of $1.220646198301469 e^{-25}$ and as such, we can conclude that the differences of usage rating is **not normally distributed**. This would entail that using the Paired T-Test is not sound and we will turn to a non-parametric alternative which is the **Wilcoxon signed rank test**.

### Wilcoxon Signed Rank Test

Acording to the book Practical Nonparametric Statistics, 3rd Edition by W. J. Conover, the Wilcoxon signed rank test is a non-parametric rank test for statistical hypothesis testing used either to test the location of a population based on a sample of data, or to compare the locations of two populations using two matched samples. To compute for the test statistic, a few steps would be covered. For the purpose of this study, we will be utilizing the matched sample aspect.

In solving, we first let $D_{i}$ be the difference of the data point from the first group, $X_{i}$, and the matching data point from the second group $Y_{i}$. The differences then are sorted and assigned ranks $R_{i}$.

$\begin{align}
T=\sum _{i=1}^{N}\operatorname {sgn}(D_{i})R_{i}.
\end{align}$

In this formula $T$ is the test statistic, ${\displaystyle \operatorname {sgn} }$ denotes the sign function: 
${\displaystyle \operatorname {sgn}(x)=1}$ if 
${\displaystyle x>0}$ and 
${\displaystyle \operatorname {sgn}(x)=-1}$ if 
${\displaystyle x<0}$.

The hypotheses is structured as follows for the paired Wilcoxon signed rank test:

> **Null hypothesis $H_{0}$**
- The observations ${\displaystyle (X_{i},Y_{i})}$ are exchangeable, meaning that ${\displaystyle (X_{i},Y_{i})}$ and ${\displaystyle (Y_{i},X_{i})}$ have the same distribution. Equivalently, ${\displaystyle F(x,y)=F(y,x)}$.


> **Two sided alternative hypothesis $H_{a}$**
- For some ${\displaystyle \mu \neq 0}$, the pairs ${\displaystyle (X_{i},Y_{i})}$ and ${\displaystyle (Y_{i}+\mu ,X_{i}-\mu )}$ have the same distribution.

Where, $ \mu $ represents the **location shift** between the two paired samples.  
- $ \mu = 0 $: no shift → distributions are the same.  
- $ \mu \ne 0 $: consistent difference in one direction.

In this analysis, the built in function from `scipy`, `wilcoxon(x, y)` will be used.
 

In [65]:
stat, p = sp.wilcoxon(usage_diff["Usage%_2015"], usage_diff["Usage%_2025"])
print(f"Wilcoxon test: stat={stat}, p={p}")
if p < 0.05:
    print("Significant difference between 2015 and 2025")
else:
    print("No significant difference between 2015 and 2025")

Wilcoxon test: stat=55928.0, p=0.004527117590067173
Significant difference between 2015 and 2025


### Results

Based on the p value, $0.00021782617076947764$, with a significance level of $0.05$, we will be rejecting the null hypothesis and there is a distribution where $\mu$ not equal to zero, that fits both the pairs ${\displaystyle (X_{i},Y_{i})}$ and ${\displaystyle (Y_{i}+\mu ,X_{i}-\mu )}$. Succintly, this states that the two groups have a **significant difference** between each other.

### Discussion

